In [1]:
import pandas as pd
import panel as pn
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import psycopg2

db_config = {
    'user': 'postgres',
    'password': '5715',
    'host': 'localhost',
    'port': '5432',
    'database': 'Sistema_de_Gereciamento_de_SaúdeFINAL'
}

engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")
con = engine.connect()


**Passo 1**
Nessa etapa foi importada as bibliotecas necessárias, dps foi armazenado a config do banco e posteriormente foi criado umn objeto de conexão com o banco e dps é esyabelecido a conexão

In [2]:
def read_encaminhamentos(id=None, CRM=None, cpf_paciente=None):
    try:
        if id is not None:
            query = text("SELECT * FROM sistema_de_saude.encaminha WHERE id = :id")
            df = pd.read_sql(query, con, params={"id": id})
        elif CRM is not None:
            query = text("SELECT * FROM sistema_de_saude.encaminha WHERE CRM = :CRM")
            df = pd.read_sql(query, con, params={"CRM": CRM})
        elif cpf_paciente is not None:
            query = text("SELECT * FROM sistema_de_saude.encaminha WHERE cpf_paciente = :cpf_paciente")
            df = pd.read_sql(query, con, params={"cpf_paciente": cpf_paciente})
        else:
            query = "SELECT * FROM sistema_de_saude.encaminha"
            df = pd.read_sql(query, con)
        
        return df
    except SQLAlchemyError as e:
        return str(e)

**passo 2**
É criado uma função que vai ler os dados da tabela e vai permitir filtrar por id, CRM do médico ou pelo cpf do paciente
- Inicia com um bloco de tentatuva que se ocorrer um erroe ele será capturado pelo except
- verifice se o id não é none, se sim ele consulta os dados pelo id
- query = text.... cria uma consulta sql que seleciona os registros da tabela a partir do id
- df = pd.read_sql.... Usa a função de leitura para executar e retornar os resultados em um dataFrame, usa o "params" pra mapear o valor id para o placeholder :id na consulta sql
- se o if estiver errado ele faz o mesmo para as outras condições
- no final ele retorna o valor do resultado, A função retorna o DataFrame df.
- except SQLAlchemyError as e:
    return str(e) Se um erro ocorrer ele é capturado e a função retorna uma string representando o erro, que pode ser útil para depuração.


In [3]:
def create_encaminhamento(CRM, data_, unidade_destino, local_destino, cpf_paciente):
    try:
        query = text("INSERT INTO sistema_de_saude.encaminha (CRM, data_, unidade_destino, local_destino, cpf_paciente) VALUES (:CRM, :data_, :unidade_destino, :local_destino, :cpf_paciente)")
        con.execute(query, {"CRM": CRM, "data_": data_, "unidade_destino": unidade_destino, "local_destino": local_destino, "cpf_paciente": cpf_paciente})
        con.commit()   
        return "Encaminhamento adicionado com sucesso!"
    except SQLAlchemyError as e:
        return str(e)

**passo 3** 
- cria uma função que inseri novos registro na tabelaCRM: O CRM do médico responsável pelo encaminhamento.
data_: A data do encaminhamento.
unidade_destino: O ID da unidade de saúde de destino.
local_destino: O nome da instituição ou local de destino.
cpf_paciente: O CPF do paciente a ser encaminhado.
- try:  inicia o bloco de tentativa
- query = text.... A função text() do SQLAlchemy é usada para preparar a consulta. A consulta INSERT INTO insere os valores fornecidos nos campos
- con.execute.... executa a consulta SQL
- con.commit() inserção dos dados no banco de dados é finalizada e confirmada. Até o commit, a inserção está pendente e pode ser revertida.
- return "Encaminhamento adicionado com sucesso!", se tudo estiver correto essa mensagem é retornada
- except SQLAlchemyError as e:
    return str(e) se ocorrer um erro, erá capturado pelo bloco except. O erro capturado é convertido em uma string (str(e)) e retornado pela função, o que permite que o erro seja exibido ou registrado para depuração.





In [4]:
def update_encaminhamento(id, CRM, data_, unidade_destino, local_destino, cpf_paciente):
    try:
        query = text("UPDATE sistema_de_saude.encaminha SET CRM = :CRM, data_ = :data_, unidade_destino = :unidade_destino, local_destino = :local_destino, cpf_paciente = :cpf_paciente WHERE id = :id")
        con.execute(query, {"id": id, "CRM": CRM, "data_": data_, "unidade_destino": unidade_destino, "local_destino": local_destino, "cpf_paciente": cpf_paciente})
        con.commit()   
        return "Encaminhamento atualizado com sucesso!"
    except SQLAlchemyError as e:
        return str(e)

**passo 4**
- atualiza os registros existentes na tabela
- inicia o try: 
- query = text... cria uma consulta SQL
- con. execute execulta a consulta
- con.commit confirma a transação
- return "Encaminhamento atualizado com sucesso!" retorno de sucesso
- except SQLAlchemyError as e:
    return str(e) se occorrer erro, captura erros relacionados ao banco de dados. Se um erro for capturado, ele é convertido em uma string com str(e) e retornado. Isso permite que a função forneça feedback sobre o erro ocorrido, facilitando a depuração.



In [5]:
def delete_encaminhamento(id):
    try:
        query = text("DELETE FROM sistema_de_saude.encaminha WHERE id = :id")
        con.execute(query, {"id": id})
        con.commit()   
        return "Encaminhamento excluído com sucesso!"
    except SQLAlchemyError as e:
        return str(e)


**passo 5**
- Responsavel por excluir um encaminhamento especifico da tabela. recebe o parametro id
- query = text.... consulta que usa o comando delete para remover um registro
- con.execute(query, {"id": id}) executa a consulta e substitui o :id pelo valor passado para  a função
- con.commit() confirma a exclusão do encaminhamento definitiva no banco de dados
- return "Encaminhamento excluído com sucesso!" se exclusão for bem sucedida
- except SQLAlchemyError as e:
    return str(e)






In [ ]:
pn.extension()

def update_table(id=None, CRM=None, cpf_paciente=None):
    df = read_encaminhamentos(id, CRM, cpf_paciente)
    table.value = df

**passo 6**
- pn.extension() ativa o panel
- def update_table responsavel por atualizar a tabela, tendo como base os filtros opcionais.
- df = read_encaminhamentos(id, CRM, cpf_paciente) chama a função  que le os dados da tabela no banco, e armazena o resultado em df, que será o datafram contendo os dados do escaminhamentos
- table.value = df Atualiza o valor do componente table com os dados da variavel df





In [7]:
def on_filter_button_click(event):
    id_filter = id_filter_input.value if id_filter_input.value else None
    CRM_filter = CRM_filter_input.value if CRM_filter_input.value else None
    cpf_paciente_filter = cpf_paciente_filter_input.value if cpf_paciente_filter_input.value else None
    update_table(id=id_filter, CRM=CRM_filter, cpf_paciente=cpf_paciente_filter)
    output_pane.object = "Tabela atualizada com base no filtro."

**passo 7**
- quando o usuário clica no botão de filtro. O argumento event é passado automaticamente pelo Panel, representando o evento de clique.
- Captura os valores dos campos de entrada (ID, CRM, CPF do paciente) preenchidos pelo usuário.
- update_table.... Atualiza a tabela de encaminhamentos com base nos filtros aplicados.
- output_pane.object = "Tabela atualizada com base no filtro." Define o conteúdo de output_pane para informar ao usuário que a tabela foi atualizada com base nos filtros



In [8]:
id_filter_input = pn.widgets.IntInput(name='Filtrar por ID', width=150, css_classes=['input-widget'])
CRM_filter_input = pn.widgets.IntInput(name='Filtrar por CRM', width=150, css_classes=['input-widget'])
cpf_paciente_filter_input = pn.widgets.TextInput(name='Filtrar por CPF do Paciente', placeholder='CPF do Paciente', width=300, css_classes=['input-widget'])
filter_button = pn.widgets.Button(name='Aplicar Filtro', button_type='primary', css_classes=['filter-button'])

filter_button.on_click(on_filter_button_click)

CRM_input = pn.widgets.IntInput(name='CRM', width=150, css_classes=['input-widget'])
data_input = pn.widgets.TextInput(name='Data', placeholder='Data do Encaminhamento', width=150, css_classes=['input-widget'])
unidade_destino_input = pn.widgets.IntInput(name='ID da Unidade de Destino', width=150, css_classes=['input-widget'])
local_destino_input = pn.widgets.TextInput(name='Local de Destino', placeholder='Nome do Local de Destino', width=300, css_classes=['input-widget'])
cpf_paciente_input = pn.widgets.TextInput(name='CPF do Paciente', placeholder='CPF do Paciente', width=300, css_classes=['input-widget'])
create_button = pn.widgets.Button(name='Adicionar Encaminhamento', button_type='primary', css_classes=['create-button'])


**passo 8**
-  Cria campos de entrada do tipo numérico. Aqui temos dois campos para filtrar por ID e CRM.
- filter_button.on_click(on_filter_button_click) Conecta o botão de filtro à função on_filter_button_click.
- Depois é definido os campos de entrada e botões que permite ao usuário filtrar os encaminhamentos existentes ou adicionar um novo encaminhamento



In [9]:
def on_create_button_click(event):
    message = create_encaminhamento(CRM_input.value, data_input.value, unidade_destino_input.value, local_destino_input.value, cpf_paciente_input.value)
    update_table()
    output_pane.object = message

    create_button.on_click(on_create_button_click)


**passo 9**
- onm_create_button_click é chamada automaticamente toda vez que o botao de encaminhamento for clicado
- message = .... chama a função create_encaminhamento para adicionar um novo encaminhamento ao banco de dados
- update_table() pós a criação do encaminhamento, a tabela de encaminhamentos exibida na interface é atualizada para refletir a nova inserção.
- output_pane.object = message Atualiza o painel de saída (output_pane) com a mensagem retornada pela função create_encaminhamento.
- create_button.on_click(on_create_button_click)  Aqui associamos a função on_create_button_click ao botão de criação de encaminhamento (create_button). Isso garante que, quando o botão for clicado, a função será executada, permitindo a criação do novo encaminhamento.




In [10]:
CRM_input = pn.widgets.IntInput(name='CRM', width=150, css_classes=['input-widget'])
data_input = pn.widgets.TextInput(name='Data', placeholder='Data do Encaminhamento', width=150, css_classes=['input-widget'])
unidade_destino_input = pn.widgets.IntInput(name='ID da Unidade de Destino', width=150, css_classes=['input-widget'])
local_destino_input = pn.widgets.TextInput(name='Local de Destino', placeholder='Nome do Local de Destino', width=300, css_classes=['input-widget'])
cpf_paciente_input = pn.widgets.TextInput(name='CPF do Paciente', placeholder='CPF do Paciente', width=300, css_classes=['input-widget'])
create_button = pn.widgets.Button(name='Adicionar Encaminhamento', button_type='primary', css_classes=['create-button'])

**passo 10**
- define os widgets da interface para inserir os dados
- CRM_input: Campo para inserir o CRM do médico (número inteiro).
- data_input: Campo para inserir a data do encaminhamento (texto).
- unidade_destino_input: Campo para inserir o ID da unidade de destino (número inteiro).
- local_destino_input: Campo para inserir o nome do local de destino (texto).
- cpf_paciente_input: Campo para inserir o CPF do paciente (texto).
- create_button: Botão para adicionar um novo encaminhamento.

In [11]:
def on_create_button_click(event):
    message = create_encaminhamento(CRM_input.value, data_input.value, unidade_destino_input.value, local_destino_input.value, cpf_paciente_input.value)
    update_table()
    output_pane.object = message

create_button.on_click(on_create_button_click)


Watcher(inst=Button(button_type='primary', css_classes=['create-button'], name='Adicionar Encaminhamento'...), cls=<class 'panel.widgets.button.Button'>, fn=<function on_create_button_click at 0x0000021CC71B14E0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

**passo 11**
- on_create_button_click(event): Função que define a ação do botão ao ser clicado. Ela aceita o evento de clique como argumento.
- message = create_encaminhamento(...): Chama a função create_encaminhamento, passando os valores inseridos nos campos (CRM_input.value, data_input.value, etc.). O retorno da função (sucesso ou erro) é armazenado na variável message.
- update_table(): Após a criação do encaminhamento, a tabela de encaminhamentos é atualizada para refletir os novos dados.
- output_pane.object = message: Exibe a mensagem de retorno (sucesso ou erro) na interface para o usuário.
- create_button.on_click(on_create_button_click): Associa a função on_create_button_click ao botão de criação (create_button), de forma que ela seja executada quando o botão for clicado.

In [12]:
id_input = pn.widgets.IntInput(name='ID do Encaminhamento', width=150, css_classes=['input-widget'])
update_CRM_input = pn.widgets.IntInput(name='Novo CRM', width=150, css_classes=['input-widget'])
update_data_input = pn.widgets.TextInput(name='Nova Data', placeholder='Nova Data', width=150, css_classes=['input-widget'])
update_unidade_destino_input = pn.widgets.IntInput(name='Nova Unidade de Destino', width=150, css_classes=['input-widget'])
update_local_destino_input = pn.widgets.TextInput(name='Novo Local de Destino', placeholder='Novo Local de Destino', width=300, css_classes=['input-widget'])
update_cpf_paciente_input = pn.widgets.TextInput(name='Novo CPF do Paciente', placeholder='Novo CPF do Paciente', width=300, css_classes=['input-widget'])
update_button = pn.widgets.Button(name='Atualizar Encaminhamento', button_type='warning', css_classes=['update-button'])

**passo 12**
- id_input: Um campo de entrada para o ID do encaminhamento que deseja ser atualizado. Usa IntInput para permitir apenas números inteiros.

- update_CRM_input: Um campo de entrada para o novo CRM do profissional de saúde. Também utiliza IntInput para garantir a entrada numérica.
- update_data_input: Um campo de texto para a nova data do encaminhamento. Utiliza TextInput, permitindo a entrada de texto (no formato de data).
- update_unidade_destino_input: Campo numérico para a nova unidade de destino, representada por um ID numérico.
- update_local_destino_input: Um campo de texto para o novo local de destino.
- update_cpf_paciente_input: Um campo de texto para inserir o novo CPF do paciente relacionado ao encaminhamento.
- update_button: Um botão que, ao ser clicado, dispara o processo de atualização do encaminhamento, utilizando a cor de alerta (warning) para indicar uma ação de modificação.



In [13]:
def on_update_button_click(event):
    message = update_encaminhamento(id_input.value, update_CRM_input.value, update_data_input.value, update_unidade_destino_input.value, update_local_destino_input.value, update_cpf_paciente_input.value)
    update_table()
    output_pane.object = message

update_button.on_click(on_update_button_click)

Watcher(inst=Button(button_type='warning', css_classes=['update-button'], name='Atualizar Encaminhamento'...), cls=<class 'panel.widgets.button.Button'>, fn=<function on_update_button_click at 0x0000021CC71B32E0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

**passo 13**
- Este código define uma ação para o botão de atualização de encaminhamentos. Quando o botão é clicado, a função on_update_button_click é chamada, capturando os valores dos campos de entrada (ID, CRM, data, unidade de destino, local de destino e CPF do paciente). Esses valores são passados para a função update_encaminhamento, que atualiza o registro no banco de dados. Após a atualização, a tabela de encaminhamentos é atualizada automaticamente, e uma mensagem de sucesso ou erro é exibida no painel output_pane. O botão de atualização é vinculado a essa função para executar a ação.

In [14]:
delete_id_input = pn.widgets.IntInput(name='ID do Encaminhamento para Excluir', width=150, css_classes=['input-widget'])
delete_button = pn.widgets.Button(name='Excluir Encaminhamento', button_type='danger', css_classes=['delete-button'])


**passo 14**
- delete_id_input: Um campo de entrada do tipo IntInput para o usuário fornecer o ID do encaminhamento que deseja excluir.
- delete_button: Um botão de ação rotulado como "Excluir Encaminhamento"

In [15]:
def on_delete_button_click(event):
    message = delete_encaminhamento(delete_id_input.value)
    update_table()
    output_pane.object = message

delete_button.on_click(on_delete_button_click)


Watcher(inst=Button(button_type='danger', css_classes=['delete-button'], name='Excluir Encaminhamento'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_delete_button_click at 0x0000021CC71B31A0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

**passo 15**
- O código define a ação de excluir um encaminhamento quando o botão de exclusão é clicado. A função on_delete_button_click é responsável por capturar o ID do encaminhamento a ser excluído, chamando a função delete_encaminhamento que realiza a exclusão no banco de dados. A tabela é atualizada automaticamente após a exclusão por meio da função update_table, e uma mensagem de sucesso ou erro é exibida no painel output_pane. A função on_delete_button_click é vinculada ao botão de exclusão através do comando delete_button.on_click, garantindo que a ação ocorra sempre que o botão for clicado.

In [16]:
table = pn.widgets.DataFrame(read_encaminhamentos(), name='Tabela de Encaminhamentos', width=600)
output_pane = pn.pane.Markdown("Mensagens aparecerão aqui", width=600)


**passo 16**
- A variável table cria um widget de DataFrame que exibe os encaminhamentos recuperados pela função read_encaminhamentos().
- A variável output_pane cria um painel Markdown que serve para exibir mensagens
- Esse painel será atualizado conforme as interações com os botões de criação, atualização ou exclusão de encaminhamentos.


In [17]:
layout = pn.Column(
    pn.pane.Markdown("# Encaminhamento", sizing_mode='stretch_width', css_classes=['title']),
    pn.Row(
        pn.Column(id_filter_input, CRM_filter_input, cpf_paciente_filter_input, filter_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(CRM_input, data_input, unidade_destino_input, local_destino_input, cpf_paciente_input, create_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(id_input, update_CRM_input, update_data_input, update_unidade_destino_input, update_local_destino_input, update_cpf_paciente_input, update_button),
        sizing_mode='stretch_width'
    ),
    pn.Row(
        pn.Column(delete_id_input, delete_button),
        sizing_mode='stretch_width'
    ),
    table,
    output_pane,
    sizing_mode='stretch_width'
)


**passo 17**
- O código define uma interface de gerenciamento de encaminhamentos usando a biblioteca Panel. Inicialmente, são criados widgets para filtrar, adicionar, atualizar e excluir encaminhamentos, incluindo campos para ID, CRM, data, unidade de destino, local e CPF do paciente, além de botões para aplicar filtros, criar, atualizar e excluir registros. A função on_filter_button_click atualiza a tabela de encaminhamentos com base nos filtros aplicados. A função on_create_button_click adiciona um novo encaminhamento e atualiza a tabela, enquanto a função on_update_button_click permite atualizar um encaminhamento existente e, por fim, a função on_delete_button_click remove um encaminhamento com base no ID fornecido. A tabela é exibida para mostrar os encaminhamentos atuais, e mensagens de status são apresentadas em um painel. O layout da interface organiza os elementos vertical e horizontalmente, garantindo que ocupem toda a largura disponível, criando uma experiência de usuário fluida e intuitiva.

In [18]:
css = """
.input-widget {
    background-color: #f0f8ff;
    border: 1px solid #007bff;
}
.filter-button {
    background-color: #007bff;
    color: white;
}
.create-button {
    background-color: #28a745;
    color: white;
}
.update-button {
    background-color: #ffc107;
    color: white;
}
.delete-button {
    background-color: #dc3545;
    color: white;
}
.title {
    color: #007bff;
    font-size: 24px;
    font-weight: bold;
}

"""

pn.config.raw_css.append(css)

layout.show()


Launching server at http://localhost:52666


**passo 18**
- definição de um estilo básico
